<a href="https://colab.research.google.com/github/DGkurnia/cnkproject/blob/main/BMLP_Derfansyah_Guswiranata_Kurnia_klasifikasi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
#Perpustakaan untuk pengelolaan data
import numpy as np
import pandas as pd
#Grafik
import matplotlib.pyplot as plt #grafik dasar
import seaborn as sns
import datetime

In [20]:
#pelabelan untuk 'onehotencoder'
from sklearn.preprocessing import OneHotEncoder as ohen

In [11]:
#Bagian svm(p1)
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import RobustScaler

In [3]:
#pengelolaan file
import os #untuk os
import time #waktu
import datetime #tanggal
import gc #pengunduhan file per batch

In [4]:
#Penyimpanan hasil sementara di google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#Bagian multiprocessing melalui instalasi pool
import warnings #peringatan
import multiprocessing #untuk multiprocessing
from multiprocessing import Pool

In [6]:
#penyimpanan hasil melalui dill
! pip install dill
import dill

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 2.7 MB/s eta 0:00:00


# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [7]:
#impor perpustakaan untuk pelabelan data
from google.colab import files
#ke proses 'upload'

In [8]:
#Tahap penguploadan
uploaded = files.upload()

Saving hslclstr.csv to hslclstr.csv


In [9]:
#Cek data
df = pd.read_csv('/content/hslclstr.csv')
dfc = df.copy() #jangan pakai data asli

In [10]:
#cek hasil
smpl = dfc.head(7)
inf = dfc.info() #informasi
desk = dfc.describe() #deskripsi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   img           64016 non-null  object 
 1   title         64016 non-null  object 
 2   console       64016 non-null  object 
 3   genre         64016 non-null  object 
 4   publisher     64016 non-null  object 
 5   total_sales   64016 non-null  float64
 6   na_sales      64016 non-null  float64
 7   jp_sales      64016 non-null  float64
 8   pal_sales     64016 non-null  float64
 9   other_sales   64016 non-null  float64
 10  critic_score  64016 non-null  float64
 11  release_date  64016 non-null  object 
 12  last_update   64016 non-null  object 
 13  Label         64016 non-null  object 
dtypes: float64(6), object(8)
memory usage: 6.8+ MB


In [12]:
#inspeksi label untuk sampel
lbl = dfc['Label'].copy()
print(lbl)

0                  PS3 Action
1                 PS4 Shooter
2        PS2 Action-Adventure
3                 X360 Sports
4           XOne Role-Playing
                 ...         
64011           GBA Adventure
64012                WS Music
64013               Wii Music
64014               Wii Music
64015                WS Music
Name: Label, Length: 64016, dtype: object


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [15]:
#Inisiasi pembagian data
inspk = dfc.iloc[:,5:11].copy() #numerikal untuk inspeksi kluster
inspk.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64016 entries, 0 to 64015
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   total_sales   64016 non-null  float64
 1   na_sales      64016 non-null  float64
 2   jp_sales      64016 non-null  float64
 3   pal_sales     64016 non-null  float64
 4   other_sales   64016 non-null  float64
 5   critic_score  64016 non-null  float64
dtypes: float64(6)
memory usage: 2.9 MB


In [16]:
#numerisasi label (identifikasi jenis label)
jnslbl = dfc['Label'].copy()
jnslbl.info()

<class 'pandas.core.series.Series'>
RangeIndex: 64016 entries, 0 to 64015
Series name: Label
Non-Null Count  Dtype 
--------------  ----- 
64016 non-null  object
dtypes: object(1)
memory usage: 500.2+ KB


In [17]:
#Sumbu x
xcls = inspk #pakai numerik

Sumbu x  **(xcls)** adalah nilai numerikal dari tabel

In [23]:
#Persiapan pembagian data (untuk kluster)
ycls = jnslbl #masih kata-kata
#pelabelan dengan 'ohen.onehotencoder' (pembentukan ulang)
bntkbaru, lbl = pd.factorize(ycls)
#inspeksi hasil
print(bntkbaru)
print(lbl)

[ 0  1  2 ...  7  7 27]
Index(['PS3 Action', 'PS4 Shooter', 'PS2 Action-Adventure', 'X360 Sports',
       'XOne Role-Playing', 'PC Simulation', 'PSP Racing', 'Wii Music',
       'PS Misc', 'DS Fighting',
       ...
       'PS Platform', 'DS Adventure', '2600 Strategy', 'GBA Puzzle', 'NES MMO',
       'XB Sandbox', 'PSN Party', 'GEN Education', 'PSV Board Game',
       'DC Visual Novel'],
      dtype='object', length=180)


In [24]:
#pembentukan ulang
enkoding = bntkbaru.reshape(-1,1)
#inisiasi ohen
encoder = ohen()
#penyesuaian data
lblohen = encoder.fit_transform(enkoding)

In [26]:
#konversi ke df untuk kluster jadi
dflbl = pd.DataFrame(lblohen.toarray(), columns=lbl)
smpllbl = dflbl.head(7)
print(smpllbl) #inspeksi hasil

   PS3 Action  PS4 Shooter  PS2 Action-Adventure  X360 Sports  \
0         1.0          0.0                   0.0          0.0   
1         0.0          1.0                   0.0          0.0   
2         0.0          0.0                   1.0          0.0   
3         0.0          0.0                   0.0          1.0   
4         0.0          0.0                   0.0          0.0   
5         0.0          0.0                   0.0          0.0   
6         0.0          0.0                   0.0          0.0   

   XOne Role-Playing  PC Simulation  PSP Racing  Wii Music  PS Misc  \
0                0.0            0.0         0.0        0.0      0.0   
1                0.0            0.0         0.0        0.0      0.0   
2                0.0            0.0         0.0        0.0      0.0   
3                0.0            0.0         0.0        0.0      0.0   
4                1.0            0.0         0.0        0.0      0.0   
5                0.0            1.0         0.0      

Nilai ini **(ycls)** menjadi komponen untuk penyatuan

# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

In [27]:
#deklarasi nilai X dan y
X_train, X_test, y_train, y_test = train_test_split(xcls, ycls, test_size=0.225, random_state=42)

In [33]:
#Fungsi penentu akurasi
def train_and_evaluate(params):
    C, gamma = params
    model = SVC(C=C, gamma=gamma)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Check if any of the metrics meet the minimum or optimal thresholds
    if (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
       (accuracy >= 0.92 and precision >= 0.92 and recall >= 0.92 and f1 >= 0.92):
        return (C, gamma, accuracy, precision, recall, f1, y_pred)
    elif (accuracy >= 0.87 and precision >= 0.87 and recall >= 0.87 and f1 >= 0.87) or \
       (accuracy < 0.92 and precision < 0.92 and recall < 0.92 and f1 < 0.92):
       print("Hasil masih layak untuk disimpan.")
       return (C, gamma, accuracy, precision, recall, f1, y_pred)
    else:
       print("Hasil tidak layak untuk disimpan.")
       return None
  #ke paralelisasi

In [34]:
#Fungsi evaluasi matriks
def generate_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    cm_rounded = np.round(cm.astype(float), decimals=4)  # Round values to four significant figures
    print("Confusion Matrix:")
    print(cm_rounded)
    return cm_rounded

Tulis narasi atau penjelasan algoritma yang Anda gunakan.
### Rincian pemakaian algoritma (Support vector machines)

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

In [ ]:
#Paralelisasi
if __name__ == '__main__':
    # Define a parameter grid with a range of gamma values
    param_grid = [(C, gamma) for C in [0.1, 1, 10] for gamma in np.linspace(0.01, 1, num=10)]

    with Pool() as pool:
        results = pool.map(train_and_evaluate, param_grid)

    # Filter out None results (those that did not meet the criteria)
    filtered_results = [result for result in results if result is not None]

    # Save results using dill
    with open('svm_results.dill', 'wb') as f:
        dill.dump(filtered_results, f)

    # Penghasilan data
    for result in filtered_results:
        _, _, _, _, _, _, y_pred = result
        cm = generate_confusion_matrix(y_test, y_pred)

        # Optionally save each confusion matrix using dill
        with open(f'confusion_matrix_C_{result[0]}_gamma_{result[1]}.dill', 'wb') as cm_file:
            dill.dump(cm, cm_file)

    # Optionally print the filtered results
    for result in filtered_results:
        print(result)

Tulis hasil evaluasi algoritma yang digunakan, jika Anda menggunakan 2 algoritma, maka bandingkan hasilnya.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

In [ ]:
#Type your code here

## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

In [ ]:
#Type your code here

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
2. Identifikasi kelemahan model, seperti:
  - Precision atau Recall rendah untuk kelas tertentu.
  - Apakah model mengalami overfitting atau underfitting?
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.